<a href="https://colab.research.google.com/github/savadikarc/vae/blob/master/vae.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
base_path = '/content/drive/My Drive/<PATH_TO_SAVE_TO>/'
!ls /content/drive/My\ Drive/DL_ML/VAE

In [ ]:
# PyTorch - 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision import datasets, transforms
import torch.nn.functional as F

# NumPy, standard
import numpy as np
from scipy.stats import norm

# Visualization
import imageio
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from keras.datasets import mnist

In [ ]:
latent_size = 2
hidden_size = 1024
base_filters = 8
batch_size = 2048
EPOCHS = 1000
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
class EncoderFC(nn.Module):
    
    def __init__(self, latent_size=10):
        
        super(EncoderFC, self).__init__()
        self.latent_size = latent_size
        
        self.fc = nn.Sequential(
            nn.Linear(784, hidden_size, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_size, hidden_size, bias=True),
            nn.ReLU(inplace=True)
        )
        
        self.linear_mu = nn.Linear(hidden_size, latent_size)
        self.linear_log_var = nn.Linear(hidden_size, latent_size)
        
    def forward(self, x):
        
        x = x.view(-1, 784)
        
        features = self.fc(x)
        mu = self.linear_mu(features)
        log_var = self.linear_log_var(features)
        
        return mu, log_var

In [ ]:
class DecoderFC(nn.Module):
    
    def __init__(self, latent_size=10):
        
        super(DecoderFC, self).__init__()
        
        self.latent_size = latent_size
        
        self.fc = nn.Sequential(
            nn.Linear(latent_size, hidden_size, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_size, hidden_size, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_size, 784)
            
        )
        
        
        self.sigmoid =  nn.Sigmoid()
        
    def forward(self, z):
        
        x = self.fc(z)
        
        x = self.sigmoid(x)
        
        return x.view(-1, 1, 28, 28)
        

In [ ]:
def weight_init(m):
    
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        m.weight.data = nn.init.kaiming_normal_(m.weight.data)
        m.weight.data.requires_grad = True
        
        try:
            m.bias.data = torch.zeros(m.bias.data.size(), requires_grad=True)
        except AttributeError:
            pass
    elif isinstance(m, nn.BatchNorm2d):
        m.weight.data = torch.ones(m.weight.data.size(), requires_grad=True)
        m.bias.data = torch.zeros(m.bias.data.size(), requires_grad=True)

In [ ]:
encoder = EncoderFC(latent_size)
encoder.apply(weight_init)
encoder = encoder.to(device)

decoder = DecoderFC(latent_size)
decoder.apply(weight_init)
decoder = decoder.to(device)

In [ ]:
def train_step(batch_X, batch_y, criterion, optimizer_e, optimizer_d):
    
    optimizer_e.zero_grad()
    optimizer_d.zero_grad()
    
    x = torch.FloatTensor(batch_X).to(device)
    labels = torch.LongTensor(batch_y)
    
    # Predictive mean and log variance
    # Data in the range of -1 to 1
    # mu, log_var = encoder((x - 0.5) / 0.5)
    mu, log_var = encoder(x)
    
    # Sample i.e. the reparameterization trick
    # var = e^log_var
    # std = sqrt(var)
    # std = e^(log(var)/2)
    z = mu + torch.randn_like(mu) * torch.exp(log_var / 2.)
    
    # Decode the sampled vector
    x_reconstructed = decoder(z)
    
    # Recostruction loss
    # Use binary crossentropy loss
    reconstruction_loss = criterion(x_reconstructed, x)
    # reconstruction_loss = criterion(x_reconstructed, (x-0.5)/0.5)
    
    # KL divergence between prior p_theta(z) over z and posterior q_phi(z|x)
    # Appendix B: Kingma and Welling, Autoencoding Variational Bayes.
    # https://arxiv.org/abs/1312.6114
    kl_divergence = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    
    loss = (reconstruction_loss + kl_divergence) / batch_X.shape[0]
    
    loss.backward()
    
    optimizer_e.step()
    optimizer_d.step()
    
    return reconstruction_loss.item(), kl_divergence.item()

In [ ]:
def train_epoch(X, y, criterion, optimizer_e, optimizer_d, batch_size=128):
    
    ptr = 0
    n_steps = X.shape[0] // batch_size + (X.shape[0]%batch_size != 0)
    
    reconstruction_loss = 0.
    kl_divergence = 0.
    for _iter in range(n_steps):
        _X, _y = X[ptr:ptr+batch_size, ...], y[ptr:ptr+batch_size]
        _reconstruction_loss, _kl_divergence = train_step(_X, _y, criterion, optimizer_e, optimizer_d)
        
        reconstruction_loss += _reconstruction_loss
        kl_divergence += _kl_divergence
        
    return reconstruction_loss / np.prod(X.shape), kl_divergence / (X.shape[0] * latent_size)

In [ ]:
def val_step(batch_X, batch_y, criterion):
    
    with torch.no_grad():
        x = torch.FloatTensor(batch_X).to(device)
        labels = torch.LongTensor(batch_y)

        # Predictive mean and log variance
        mu, log_var = encoder(x)

        # Sample i.e. the reparameterization trick
        # var = e^log_var
        # std = sqrt(var)
        # std = e^(log(var)/2)
        z = mu + torch.randn_like(mu) * torch.exp(log_var / 2.)

        # Decode the sampled vector
        x_reconstructed = decoder(z)

        # Recostruction loss
        # Use binary crossentropy loss
        reconstruction_loss = criterion(x_reconstructed, x)
        # reconstruction_loss = criterion(x_reconstructed, (x-0.5)/0.5)

        # KL divergence between prior p_theta(z) over z and posterior q_phi(z|x)
        # Appendix B: Kingma and Welling, Autoencoding Variational Bayes.
        # https://arxiv.org/abs/1312.6114
        kl_divergence = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    
    return reconstruction_loss.item(), kl_divergence.item()

In [ ]:
def val_epoch(X, y, criterion, batch_size=128):
    
    ptr = 0
    n_steps = X.shape[0] // batch_size + (X.shape[0]%batch_size != 0)
    
    reconstruction_loss = 0.
    kl_divergence = 0.
    for _iter in range(n_steps):
        _X, _y = X[ptr:ptr+batch_size, ...], y[ptr:ptr+batch_size]
        _reconstruction_loss, _kl_divergence = val_step(_X, _y, criterion)
        
        reconstruction_loss += _reconstruction_loss
        kl_divergence += _kl_divergence
        
    return reconstruction_loss / np.prod(X.shape), kl_divergence / (X.shape[0] * latent_size)

In [ ]:
def visualize(noise):
    
    """noise: torch Tensor
    """
    
    n_images = noise.size(0)
    
    rows = int(np.sqrt(n_images))
    cols = rows
    
    grid = np.zeros((rows*28, cols*28), dtype=np.uint8)
    
    with torch.no_grad():
        _x = decoder(noise)
        
    images = _x.cpu().numpy()
    
    ptr = 0
    for i in range(rows):
        _row = i * 28
        for j in range(cols):
            _col = j * 28
            img = images[ptr]
            if img.shape[0] == 1:
                img = np.squeeze(img, axis=0)
            else:
                img = np.transpose(img, axes=(1, 2, 0))
            img = (img * 255.).astype(np.uint8)
            grid[_row:_row+28, _col:_col+28] = img
            ptr += 1
            
    cmap = 'gray' if np.ndim(img) == 2 else None
    plt.imshow(grid, cmap=cmap)
    plt.show()

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
X_train = 1. - np.expand_dims(X_train, axis=1) / 255.
X_test = 1. - np.expand_dims(X_test, axis=1) / 255.

In [ ]:
fixed_noise_vector = torch.FloatTensor(64, latent_size).normal_(0., 1.)
if 'cuda' in device:
    fixed_noise_vector = fixed_noise_vector.to(device)

In [ ]:
optimizer_e = optim.Adam(encoder.parameters(), lr=1e-4)
optimizer_d = optim.Adam(decoder.parameters(), lr=1e-4)

In [ ]:
criterion = nn.BCELoss(reduction='sum')

In [ ]:
for epoch in range(EPOCHS):

    train_indices = np.random.permutation(X_train.shape[0])
    _X_train = X_train[train_indices]
    _y_train = y_train[train_indices]

    test_indices = np.random.permutation(X_test.shape[0])
    _X_test = X_test[test_indices]
    _y_test = y_test[test_indices]
    
    train_reconstruction_loss, train_kl_divergence = train_epoch(_X_train, _y_train, criterion, optimizer_e, optimizer_d, batch_size=batch_size)
    print('Train: Epoch: {} | BCE: {:.5f} | KL Divergence: {:.5f}'.format(epoch, train_reconstruction_loss, train_kl_divergence))
    val_reconstruction_loss, val_kl_divergence = val_epoch(_X_test, _y_test, criterion, batch_size)
    print('Val: Epoch: {} | BCE: {:.5f} | KL Divergence: {:.5f}'.format(epoch, val_reconstruction_loss, val_kl_divergence))
    
    if epoch % 10 == 0:
        visualize(fixed_noise_vector)

In [ ]:
dec = decoder.eval().to(device)
enc = encoder.eval().to(device)

In [ ]:
def interpolate(X, y, from_digit=1, to_digit=0, steps=30, enc=None, dec=None):
    """
    Interpolate from one digit to another
    """
    idx = int(np.random.randint(0, 100))
    X_from = X[y == from_digit][52]
    X_to = X[y == to_digit][52]
    
    X_from = torch.FloatTensor(np.expand_dims(X_from, axis=0)).to(device)
    X_to = torch.FloatTensor(np.expand_dims(X_to, axis=0)).to(device)
    
    mu_from, log_var_from = enc(X_from)
    mu_to, log_var_to = enc(X_to)
    
    z_from = mu_from + torch.randn_like(mu_from) * torch.exp(log_var_from / 2.)
    z_to = mu_to + torch.randn_like(mu_to) * torch.exp(log_var_to / 2.)
    
    grid = np.zeros((28, 28*steps), dtype=np.uint8)
    
    images = []
    for eps in np.linspace(0., 1., steps):
        z = z_from + (z_to - z_from) * eps
        image = dec(z.unsqueeze(0))
        images.append(image.detach().cpu().numpy())
        
    for i, img in enumerate(images):
        _img = img[0] * 255.
        _img = _img.astype(np.uint8)
        
        grid[:, i*28:i*28+28] = _img
        
    f = plt.figure(figsize=(20, 20))
    plt.imshow(grid, cmap='gray')
    plt.axis('off')
    plt.show()

    return images

In [ ]:
def interpolate_multiple(X, y, from_digit=1, to_digit1=0, to_digit2=7, steps=10, enc=None, dec=None):
    """
    Interpolate from one digit to another, then to another
    """
    X_from = X[y == from_digit][8] # Any random index
    X_to1 = X[y == to_digit1][89]
    X_to2 = X[y == to_digit2][56]
    
    X_from = torch.FloatTensor(np.expand_dims(X_from, axis=0)).to(device)
    X_to1 = torch.FloatTensor(np.expand_dims(X_to1, axis=0)).to(device)
    X_to2 = torch.FloatTensor(np.expand_dims(X_to2, axis=0)).to(device)
    
    mu_from, log_var_from = enc(X_from)
    mu_to1, log_var_to1 = enc(X_to1)
    mu_to2, log_var_to2 = enc(X_to2)
    
    z_from = mu_from + torch.randn_like(mu_from) * torch.exp(log_var_from / 2.)
    z_to1 = mu_to1 + torch.randn_like(mu_to1) * torch.exp(log_var_to1 / 2.)
    z_to2 = mu_to2 + torch.randn_like(mu_to2) * torch.exp(log_var_to2 / 2.)
    
    grid = np.zeros((28, 28*steps*2 - 28), dtype=np.uint8)
    
    images = []
    for eps in np.linspace(0., 1., steps):
        z1 = z_from + (z_to1 - z_from) * eps
        
        image = dec(z1.unsqueeze(0))
        images.append(image.detach().cpu().numpy())
        
    for idx, eps in enumerate(np.linspace(0., 1., steps)):
        if idx == 0:
            continue
        z2 = (z_to1 + (z_to2 - z_to1) * eps)
        image = dec(z2.unsqueeze(0))
        images.append(image.detach().cpu().numpy())

    for i, img in enumerate(images):
        _img = img[0] * 255.
        _img = _img.astype(np.uint8)
        
        grid[:, i*28:i*28+28] = _img
        
    f = plt.figure(figsize=(20, 20))
    plt.imshow(grid, cmap='gray')
    plt.axis('off')
    plt.show()

    return images

In [ ]:
def interpolate_multiple2(X, y, from_digit=1, to_digit1=0, to_digit2=7, steps=10, enc=None, dec=None):
    """
    Interpolate from one digit to another, then to another, and back to the starting digit
    """

    X_from = X[y == from_digit][80] # Any random index
    X_to1 = X[y == to_digit1][72]
    X_to2 = X[y == to_digit2][67]
    
    X_from = torch.FloatTensor(np.expand_dims(X_from, axis=0)).to(device)
    X_to1 = torch.FloatTensor(np.expand_dims(X_to1, axis=0)).to(device)
    X_to2 = torch.FloatTensor(np.expand_dims(X_to2, axis=0)).to(device)
    
    mu_from, log_var_from = enc(X_from)
    mu_to1, log_var_to1 = enc(X_to1)
    mu_to2, log_var_to2 = enc(X_to2)
    
    z_from = mu_from + torch.randn_like(mu_from) * torch.exp(log_var_from / 2.)
    z_to1 = mu_to1 + torch.randn_like(mu_to1) * torch.exp(log_var_to1 / 2.)
    z_to2 = mu_to2 + torch.randn_like(mu_to2) * torch.exp(log_var_to2 / 2.)
    
    grid = np.zeros((28, 28*steps*3 - 2*28), dtype=np.uint8)
    
    images = []
    for eps in np.linspace(0., 1., steps):
        z1 = z_from + (z_to1 - z_from) * eps
        
        image = dec(z1.unsqueeze(0))
        images.append(image.detach().cpu().numpy())
        
    for idx, eps in enumerate(np.linspace(0., 1., steps)):
        if idx == 0:
            continue
        z2 = (z_to1 + (z_to2 - z_to1) * eps)
        image = dec(z2.unsqueeze(0))
        images.append(image.detach().cpu().numpy())

    for idx, eps in enumerate(np.linspace(0., 1., steps)):
        if idx == 0:
            continue
        z4 = (z_to2 + (z_from - z_to2) * eps)
        image = dec(z4.unsqueeze(0))
        images.append(image.detach().cpu().numpy())

    for i, img in enumerate(images):
        _img = img[0] * 255.
        _img = _img.astype(np.uint8)
        
        grid[:, i*28:i*28+28] = _img
        
    f = plt.figure(figsize=(20, 20))
    plt.imshow(grid, cmap='gray')
    plt.axis('off')
    plt.show()

    return images

In [ ]:
for i in [9]:
    imgs = interpolate(X_test, y_test, from_digit=1, to_digit=i, steps=150, enc=enc, dec=dec)
    # Uncomment to save as a gif
    # imageio.mimwrite(base_path + 'b_interpolation_{}to{}.gif'.format(1, i), [(ing[0, 0, :, :] * 255.).astype(np.uint8) for ing in imgs], fps=15)
    # imageio.mimwrite(base_path + 'w_interpolation_{}to{}.gif'.format(1, i), [((1. - ing[0, 0, :, :]) * 255.).astype(np.uint8) for ing in imgs], fps=15)

In [ ]:
imgs = interpolate_multiple(X_test, y_test, from_digit=4, to_digit1=9, 
                            to_digit2=1, steps=25, 
                            enc=enc, dec=dec)
# Uncomment to save as gif
# imageio.mimwrite(base_path + 'vae_interpolation_loop.gif', [(imgs[0, 0, :, :] * 255.).astype(np.uint8) for img in imgs], fps=15)

In [ ]:
imgs2 = interpolate_multiple2(X_test, y_test, from_digit=1, to_digit1=0, 
                            to_digit2=9, steps=75, 
                            enc=enc, dec=dec)
# Uncomment to save as gif
# imageio.mimwrite(base_path + 'vae_interpolation_loop_v5.gif', [(ing[0, 0, :, :] * 255.).astype(np.uint8) for ing in imgs2], fps=10)

# Jitter Experiments:
See what happens when we perturb one of the latent dimensions 

In [ ]:
def perturb(X, y, steps=10, dec=None, save=False):
    """
    We can generate the entire grid as a batch. This is not feasible for large
    number of steps
    """
    # Generate samples from the support of p(z)
    x = norm.ppf(np.linspace(0.05, 0.95, num=steps))
    y = norm.ppf(np.linspace(0.05, 0.95, num=steps))
    l_x = x.shape[0]
    l_y = y.shape[0]
    x_grid = np.repeat(x, l_y).reshape(-1, 1)
    y_grid = np.tile(y, l_x).reshape(-1, 1)
    _z = np.concatenate([x_grid, y_grid], axis=1)

    z = torch.from_numpy(_z).float().to(device)
    
    grid = np.zeros((28*x.shape[0], 28*y.shape[0]), dtype=np.uint8)
    
    images = dec(z).detach().cpu().numpy()
    
    ptr = 0
    for i, _x in enumerate(x):
        for j, _y in enumerate(y):
        
            img = images[ptr, 0, ...]
            _img = img * 255.
            _img = _img.astype(np.uint8)
            
            grid[i*28:i*28+28, j*28:j*28+28] = _img

            ptr += 1
        
    f = plt.figure(figsize=(10, 10))
    plt.imshow(grid, cmap='gray')
    plt.axis('off')
    plt.show()

    if save:
        grid_img = Image.fromarray(grid)
        grid_img.save(base_path + 'vae_mnist_manifold.png')
        grid_img.close()

In [ ]:
def perturb_iterative(X, y, steps=10, dec=None, save=False):
    """
    For a large number of steps, generate the grid iteratively
    """
    # Generate samples from the support of p(z)
    x = norm.ppf(np.linspace(0.05, 0.95, num=steps))
    y = norm.ppf(np.linspace(0.05, 0.95, num=steps))
    l_x = x.shape[0]
    l_y = y.shape[0]
    x_grid = np.repeat(x, l_y).reshape(-1, 1)
    y_grid = np.tile(y, l_x).reshape(-1, 1)
    _z = np.concatenate([x_grid, y_grid], axis=1)
    
    grid = np.zeros((28*x.shape[0], 28*y.shape[0]), dtype=np.uint8)
    n_rows, n_cols = grid.shape

    images_l = []
    for k in range(_z.shape[0]):
        z = torch.from_numpy(_z[k]).unsqueeze(dim=0).float().to(device)
        images_l.append(dec(z).detach().cpu().numpy())
    
    ptr = 0
    for i, _x in enumerate(x): # -1, -1, ..0, 0, 
        for j, _x in enumerate(x): # -1, 0, 1,
        
            img = images_l[ptr][0, ...]
            _img = img * 255.
            _img = _img.astype(np.uint8)
            
            grid[n_rows-(j*28+28):n_rows-j*28, i*28:i*28+28] = _img

            ptr += 1
        
    f = plt.figure(figsize=(10, 10))
    plt.imshow(grid, cmap='gray')
    plt.axis('off')
    plt.show()

    if save:
        grid_img = Image.fromarray(grid)
        grid_img.save(base_path + 'vae_mnist_manifold_{}_v3.png'.format(steps))
        grid_img.close()

In [ ]:
perturb_iterative(X_test, y_test, steps=10, dec=decoder, save=False)

In [ ]:
def plot_latent(X, y, enc, samples_per_class=10):

    """
    Plot the transformed samples from the test set
    """

    fig = plt.figure(figsize=(10, 10))

    for digit in range(10):

        # Draw 10 random samples
        _X = X[y == digit]
        _y = y[y == digit]
        indices = np.random.permutation(np.arange(_X.shape[0]))
        _X = _X[indices[:samples_per_class]]

        X_sample = torch.from_numpy(_X).float().to(device)

        mu, log_var = enc(X_sample)
        z = (mu + torch.randn_like(mu) * torch.exp(log_var / 2.)).detach().cpu().numpy()

        plt.scatter(z[:, 0], z[:, 1], c=None, alpha=0.7)
    plt.legend([_ for _ in range(10)])
    plt.xlabel('z1')
    plt.ylabel('z2')
    # plt.savefig(base_path + '/vae_latent_plot_v3.png', dpi=300)
    plt.show()


In [ ]:
# Plot the transformed samples from the test set
plot_latent(X_test, y_test, encoder, samples_per_class=300)